In [33]:
using CSV
using DataFrames

In [34]:
df = DataFrame()
origin_path = "./original"  # Replace with your directory path

# List all files and directories
for year in readdir(origin_path)
    year_path = joinpath(origin_path, year)
    if isdir(year_path)
        for page in readdir(year_path)
            page_path = joinpath(year_path, page)
            if isdir(page_path)
                file = joinpath(page_path, "tracks.csv")
                single = CSV.File("./original/2023/1/tracks.csv") |> DataFrame
                append!(df, single)
            end
        end
    end
end

first(df, 5)


Row,num,title,duration,url,composer_urls,composer_names,performer_urls,performer_names,album_url
,Int64,String,String?,String,String?,String?,String,String,String
1,1,Praise Our Lord All Ye Gentiles,02:46,https://www.allmusic.com/performance/praise-our-lord-all-ye-gentiles-mq0002563548,https://www.allmusic.com/artist/william-byrd-mn0000804200,William Byrd,https://www.allmusic.com/artist/fretwork-mn0000505763;https://www.allmusic.com/artist/kings-singers-mn0000929562,Fretwork;King's Singers,https://www.allmusic.com/album/tom-will-weelkes-byrd-400-years-mw0003874125
2,2,If Women Could Be Fair,03:41,https://www.allmusic.com/performance/if-women-could-be-fair-mq0002563549,https://www.allmusic.com/artist/william-byrd-mn0000804200,William Byrd,https://www.allmusic.com/artist/fretwork-mn0000505763;https://www.allmusic.com/artist/kings-singers-mn0000929562,Fretwork;King's Singers,https://www.allmusic.com/album/tom-will-weelkes-byrd-400-years-mw0003874125
3,3,"Say Dear, When Will Your Frowning Leave",02:46,https://www.allmusic.com/performance/say-dear-when-will-your-frowning-leave-mq0002563550,https://www.allmusic.com/artist/thomas-weelkes-mn0001596008,Thomas Weelkes,https://www.allmusic.com/artist/fretwork-mn0000505763;https://www.allmusic.com/artist/kings-singers-mn0000929562,Fretwork;King's Singers,https://www.allmusic.com/album/tom-will-weelkes-byrd-400-years-mw0003874125
4,4,Like Two Proud Armies,01:41,https://www.allmusic.com/performance/like-two-proud-armies-mq0002563551,https://www.allmusic.com/artist/thomas-weelkes-mn0001596008,Thomas Weelkes,https://www.allmusic.com/artist/fretwork-mn0000505763;https://www.allmusic.com/artist/kings-singers-mn0000929562,Fretwork;King's Singers,https://www.allmusic.com/album/tom-will-weelkes-byrd-400-years-mw0003874125
5,5,Pavan No. 3,02:41,https://www.allmusic.com/performance/pavan-no-3-mq0002563552,https://www.allmusic.com/artist/thomas-weelkes-mn0001596008,Thomas Weelkes,https://www.allmusic.com/artist/fretwork-mn0000505763;https://www.allmusic.com/artist/kings-singers-mn0000929562,Fretwork;King's Singers,https://www.allmusic.com/album/tom-will-weelkes-byrd-400-years-mw0003874125


In [35]:
# Define a function to extract ID from a given URL
function extract_id_from_url(url::String)
    m = match(r"([a-z]+\d+)$", url)
    return m === nothing ? "unknown" : m.captures[1]
end

# Create an empty DataFrame for Composers with the specified column names
composers = DataFrame("Composer ID" => String[], "Composer Name" => String[], "URL" => String[])

# Populate the Composers DataFrame
for i in 1:nrow(df)
    # Check for missing values before splitting
    composer_urls = ismissing(df[i, :composer_urls]) ? [] : split(df[i, :composer_urls], ";")
    composer_names = ismissing(df[i, :composer_names]) ? [] : split(df[i, :composer_names], ";")
    
    for composer_url in composer_urls, composer_name in composer_names
        composer_id = extract_id_from_url(String(composer_url))
        push!(composers, [composer_id, composer_name, composer_url])
    end
end

# Remove duplicate entries
composers = unique!(composers)
size(composers)

(778, 3)

In [36]:
# Create an empty DataFrame for Performers
performers = DataFrame("Performer ID" => String[], "Performer Name" => String[], "URL" => String[])

# Populate the Performers DataFrame
for i in 1:nrow(df)
    for performer_url in split(df[i, :performer_urls], ";"), performer_name in split(df[i, :performer_names], ";")
        performer_id = extract_id_from_url(String(performer_url))  # Convert SubString to String here
        push!(performers, [performer_id, performer_name, performer_url])
    end
end

# Remove duplicate entries
performers = unique!(performers);
size(performers)

(83, 3)

In [37]:
# Create an empty DataFrame for mapping Composers to Tracks
composers_tracks_map = DataFrame("Composer ID" => String[], "Track ID" => String[])

# Populate the mapping DataFrame
for i in 1:nrow(df)
    track_id = extract_id_from_url(df[i, :url])

    composer_urls = ismissing(df[i, :composer_urls]) ? [] : split(df[i, :composer_urls], ";")
    for composer_url in composer_urls
        composer_id = extract_id_from_url(String(composer_url))  # Convert SubString to String here
        push!(composers_tracks_map, [composer_id, track_id])
    end
end

size(composers_tracks_map)

(5340, 2)

In [38]:
# Create an empty DataFrame for mapping Performers to Tracks
performers_tracks_map = DataFrame("Performer ID" => String[], "Track ID" => String[])

# Populate the mapping DataFrame
for i in 1:nrow(df)
    track_id = extract_id_from_url(df[i, :url])
    for performer_url in split(df[i, :performer_urls], ";")
        performer_id = extract_id_from_url(String(performer_url))  # Convert SubString to String here
        push!(performers_tracks_map, [performer_id, track_id])
    end
end

size(performers_tracks_map)


(6114, 2)

In [40]:
CSV.write("wrangled/composers.csv", composers)
CSV.write("wrangled/performers.csv", performers)
CSV.write("wrangled/composers_tracks_map.csv", composers_tracks_map)
CSV.write("wrangled/performers_tracks_map.csv", performers_tracks_map);
